## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-14-44-27 +0000,bbc,Unite votes to suspend Rayner's membership ove...,https://www.bbc.com/news/articles/cx24de0d9rdo
1,2025-07-11-14-39-16 +0000,bbc,Ex-MP faces two-year wait for betting offences...,https://www.bbc.com/news/articles/c056v0m56pyo
2,2025-07-11-14-37-29 +0000,bbc,Wafcon return provides 'hope' in eastern DR Congo,https://www.bbc.com/sport/football/articles/cg...
3,2025-07-11-14-19-11 +0000,nyt,Republicans Blame Canada for Wildfire Smoke ‘S...,https://www.nytimes.com/2025/07/11/world/canad...
4,2025-07-11-14-18-56 +0000,bbc,US cuts visa validity for applicants from four...,https://www.bbc.com/news/articles/cx23ywg9w82o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
98,trump,12
107,brazil,5
25,canada,3
109,bolsonaro,3
16,wafcon,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,32
40,2025-07-10-20-10-28 +0000,nyt,"Targeting Brazil, Trump Tests Legal Limit of H...",https://www.nytimes.com/2025/07/10/us/politics...,25
44,2025-07-10-18-19-52 +0000,bbc,Why is Trump targeting Brazil - and will it ba...,https://www.bbc.com/news/articles/crk68drj57mo,25
39,2025-07-10-20-22-02 +0000,nyt,US-Brazil Tariffs: What to Know About Trump’s ...,https://www.nytimes.com/2025/07/10/world/ameri...,24
16,2025-07-11-11-21-26 +0000,nyt,Behind Trump’s Decision to Tax Brazil to Save ...,https://www.nytimes.com/2025/07/11/world/ameri...,24


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,32,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
24,18,2025-07-11-08-06-08 +0000,bbc,UK-France migrant deal 'robust' against legal ...,https://www.bbc.com/news/articles/cx24d70gw41o
20,15,2025-07-11-09-30-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
1,13,2025-07-11-14-39-16 +0000,bbc,Ex-MP faces two-year wait for betting offences...,https://www.bbc.com/news/articles/c056v0m56pyo
11,13,2025-07-11-12-43-08 +0000,bbc,Gaza's largest functioning hospital facing dis...,https://www.bbc.com/news/articles/cdx5zeywgrgo
22,13,2025-07-11-09-00-37 +0000,nyt,"In ‘Apocalypse in the Tropics,’ Director Petra...",https://www.nytimes.com/2025/07/11/movies/petr...
15,11,2025-07-11-11-43-52 +0000,nyt,Good Vibes in Trump Trade Talks Haven’t Shield...,https://www.nytimes.com/2025/07/11/world/canad...
37,11,2025-07-10-21-44-33 +0000,bbc,Nigeria win to book spot in Wafcon quarter-finals,https://www.bbc.com/sport/football/articles/cg...
7,10,2025-07-11-13-34-12 +0000,bbc,Iran could recover some enriched uranium after...,https://www.bbc.com/news/articles/cn0qzxvydz1o
42,10,2025-07-10-19-45-28 +0000,bbc,Watch: Rare shelf cloud seen over Maryland ami...,https://www.bbc.com/news/videos/cwyg93lzqd7o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
